In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
dataset = spark.read.format('csv').load('gs://hkanjih-spark-udemy/material/Spark_for_Machine_Learning/Clustering/seeds_dataset.csv', header=True, inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [5]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [9]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [11]:
final_data = assembler.transform(dataset)

In [12]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler =StandardScaler(inputCol='features',outputCol='scaledFeatures')

In [15]:
scaler_model = scaler.fit(final_data)

In [16]:
final_data = scaler_model.transform(final_data)

In [17]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [18]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [19]:
model = kmeans.fit(final_data)

In [20]:
print('WSSE')
print(model.computeCost(final_data))

WSSE
429.075596715


In [21]:
print(model.clusterCenters())

[array([  6.31670546,  12.37109759,  37.39491396,  13.91155062,
         9.748067  ,   2.39849968,  12.2661748 ]), array([  4.87257659,  10.88120146,  37.27692543,  12.3410157 ,
         8.55443412,   1.81649011,  10.32998598]), array([  4.06105916,  10.13979506,  35.80536984,  11.82133095,
         7.50395937,   3.27184732,  10.42126018])]


In [22]:
model.transform(final_data).show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|            features|      scaledFeatures|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|[5.24452795332028...|         1|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|[5.11393027165175...|         1|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|[4.91116018695588...|         1|
|13.84|    13.94|     0.8955